In [3]:
from os import path
import argparse
from os import path
import sys
import json
sys.path.append(path.abspath('eo-metadata-tools/CMR/python'))
import cmr.auth.token as t
import cmr.search.collection as coll
import cmr.search.common as scom
import cmr.util.network as net

In [ ]:
def mySearch(env, search_type, query, show=False):
    ''' Constructs a CMR search URL based on three inputs: 
        the environment, type of search, and the query syntax.
        Fourth optional argument will print out the search URL if True. 
        Leverages the eo-metadata-tools python package to submit the
        request, download the results, and return them to the calling module.
    '''
    # CMR base URLs
    cmr_root = {'PROD': 'https://cmr.earthdata.nasa.gov',
                'UAT': 'https://cmr.uat.earthdata.nasa.gov',
                'SIT': 'https://cmr.sit.earthdata.nasa.gov'}
    # Construct the search URL using the inputs
    base = cmr_root[env]
    url = '{}/{}?{}'.format(base, search_type, net.expand_query_to_parameters(query))
    if show:
        print(url)
    try:
        result = net.get(url, accept="application/vnd.nasa.cmr.umm_results+json")
        #items = result.get("items", [])
        return result
    except:
        return False

In [ ]:
def getCollMeta(cname, env, daac='GES_DISC'):
    ''' Returns a collection's metadata record.
        Input cname has syntax: shortname_version 
        Input env is one of these: SIT, UAT, or PROD
        Input daac is 'GES_DISC' by default
    '''
    return mySearch(env, 'collections', {'native-id':cname,'provider':daac})

In [ ]:
def getCollSvcs(coll, env, daac='GES_DISC'):
    ''' Searches for a collection's associated services.
        Inputs are the collection metadata record and the env.
        Returns the concept_ids and names.
    '''
    try:
        svcL = coll['meta']['associations']['services']
        svcRecs = []
        for svc in svcL: 
            sitems = mySearch(env, 'services', {'provider':daac, 'concept_id':svc})
            svcRecs.append(sitems)
    except:
        svcRecs = []
    return svcRecs